# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846638399355                   -0.70    4.8
  2   -7.852295086911       -2.25       -1.53    1.0
  3   -7.852608513631       -3.50       -2.56    1.5
  4   -7.852645982812       -4.43       -2.87    2.8
  5   -7.852646468307       -6.31       -3.12    1.0
  6   -7.852646679586       -6.68       -4.11    1.0
  7   -7.852646686672       -8.15       -4.92    2.0
  8   -7.852646686719      -10.33       -5.35    1.2
  9   -7.852646686724      -11.32       -5.38    1.5
 10   -7.852646686730      -11.25       -6.85    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846841347347                   -0.70           4.5
  2   -7.852528543113       -2.25       -1.63   0.80    2.2
  3   -7.852637421733       -3.96       -2.71   0.80    1.0
  4   -7.852646499899       -5.04       -3.31   0.80    2.0
  5   -7.852646682019       -6.74       -4.19   0.80    2.0
  6   -7.852646686619       -8.34       -4.82   0.80    1.8
  7   -7.852646686721       -9.99       -5.67   0.80    1.5
  8   -7.852646686730      -11.06       -6.53   0.80    2.2


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.414940e+01     3.120121e+00
 * time: 0.45061206817626953
     1     1.420561e+00     1.763995e+00
 * time: 0.6467819213867188
     2    -1.782892e+00     2.118194e+00
 * time: 0.6735239028930664
     3    -3.919125e+00     1.794121e+00
 * time: 0.7121329307556152
     4    -5.551209e+00     1.501207e+00
 * time: 0.7504940032958984
     5    -6.982687e+00     8.042364e-01
 * time: 0.7890100479125977
     6    -7.237075e+00     8.995277e-01
 * time: 0.8154439926147461
     7    -7.516912e+00     6.262793e-01
 * time: 0.8420989513397217
     8    -7.634263e+00     4.498298e-01
 * time: 0.8689699172973633
     9    -7.737120e+00     1.439591e-01
 * time: 0.8957018852233887
    10    -7.794281e+00     1.416820e-01
 * time: 0.9221148490905762
    11    -7.827572e+00     5.142315e-02
 * time: 0.9490408897399902
    12    -7.839128e+00     3.449603e-02
 * time: 0.9753868579864502
    13    -7.844519e+00     3.530756e-02
 * time: 1.001791954

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846875934561                   -0.70    4.5


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645913179                   -1.64
  2   -7.852646686730       -6.11       -3.71
  3   -7.852646686730      -13.32       -7.25


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.6257895681386296e-7
|ρ_newton - ρ_scfv| = 1.4181820098016762e-7
|ρ_newton - ρ_dm|   = 0.018353978395237024
